In [49]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
data = pd.read_csv('../Dataset/vagalume.csv')

In [51]:
def limpa_texto(texto):
    novo_texto = str(texto).lower()
    novo_texto = re.sub(r'\W', ' ', novo_texto)
    novo_texto = re.sub(r'\s+', ' ', novo_texto)
    return novo_texto

In [52]:
novo_data = data['music_lyric'].apply(limpa_texto).to_list()

In [53]:
novo_data

['eu tô trazendo a novidade em salvador eu tô trazendo a novidade em salvador o robsão traz uma dança pro verão o robsão traz uma dança pro verão õô pegue a latinha e amasse com o bumbum pegue a latinha e amasse com o bumbum então amasse a latinha com a bunda amasse a latinha com a bunda amasse a latinha com a bunda senta senta amasse a latinha com a bunda amasse a latinha com a bunda amasse a latinha com a bunda senta mas a latinha é da schin ou da skol brahma bavaria antartica ou sol pegue a latinha ou então pegue o latão agora senta senta senta na pressão então amasse a latinha com a bunda amasse a latinha com a bunda amasse a latinha com a bunda senta senta então amasse a latinha com a bunda amasse a latinha com a bunda amasse a latinha com a bunda senta então amasse a latinha com a bunda amasse a latinha com a bunda amasse a latinha com a bunda senta senta amasse a latinha com a bunda amasse a latinha com a bunda amasse a latinha com a bunda senta repete ',
 'você marcou comigo pr

In [55]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(novo_data).toarray()